In [1]:
import os
import torch
import pandas as pd
import scanpy as sc

In [3]:
import SpatialGlue

In [5]:
from scipy.spatial import cKDTree
from scipy.stats import mode
def cKD_refine_label(coords, labels, k):
    # Step 1: Build KD-Tree
    tree = cKDTree(coords.copy())

    # Step 2: Find k-nearest neighbors for each spot
    # k+1 because the closest point is itself
    distances, neighbors = tree.query(coords, k=k+1)

    # Exclude self-neighbor (first column)
    neighbors = neighbors[:, 1:]

    # Step 3: Reassign labels
    new_labels = labels.copy()
    for i, nbrs in enumerate(neighbors):
        # Get the labels of neighboring spots
        neighbor_labels = labels[nbrs]
        # Find the most common label among neighbors
        most_common_label = mode(neighbor_labels, keepdims=True).mode[0]
        # Reassign the label
        new_labels[i] = most_common_label
    return (new_labels)

In [7]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
from numpy.random import default_rng
import scanpy as sc
# import squidpy as sq
from anndata import AnnData
import scipy
# sc.logging.print_header()
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
import pandas as pd
import seaborn as sns
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
from sklearn.metrics.cluster import normalized_mutual_info_score, homogeneity_score, completeness_score
from numpy import genfromtxt

In [11]:
file_fold = '/Users/melancholy/Desktop/SpatialGlue/Data_SpatialGlue/Dataset7_Mouse_Brain_ATAC' 

adata_omics1 = sc.read_h5ad(file_fold + '/adata_RNA.h5ad')
adata_omics2 = sc.read_h5ad(file_fold + '/adata_peaks_normalized.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()

In [13]:
adata_omics1, adata_omics2

(AnnData object with n_obs × n_vars = 9215 × 22914
     obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_peaks', 'nFeature_peaks', 'RNA_clusters', 'ATAC_clusters'
     var: 'name'
     obsm: 'X_pca', 'X_umap', 'spatial',
 AnnData object with n_obs × n_vars = 9215 × 121068
     obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_peaks', 'nFeature_peaks', 'RNA_clusters', 'ATAC_clusters'
     var: 'count', 'percentile'
     uns: 'ATAC', 'ATAC_clusters_colors', 'umap'
     obsm: 'X_lsi', 'X_pca', 'X_umap', 'spatial'
     obsp: 'ATAC_connectivities', 'ATAC_distances')

## SpatialGlue

In [15]:
# Specify data type
data_type = '10x'
# Fix random seed
from SpatialGlue.preprocess import fix_seed
random_seed = 2022
fix_seed(random_seed)

In [17]:
from SpatialGlue.preprocess import clr_normalize_each_cell, pca, lsi

In [19]:
# RNA
sc.pp.filter_genes(adata_omics1, min_cells=10)
sc.pp.filter_cells(adata_omics1, min_genes=200)

# sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1)

# adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1, n_comps=50)

In [23]:
# ATAC
adata_omics2 = adata_omics2[adata_omics1.obs_names].copy() # .obsm['X_lsi'] represents the dimension reduced feature
if 'X_lsi' not in adata_omics2.obsm.keys():
    # sc.pp.highly_variable_genes(adata_omics2, flavor="seurat_v3", n_top_genes=3000)
    lsi(adata_omics2, use_highly_variable=False, n_components=51)

adata_omics2.obsm['feat'] = adata_omics2.obsm['X_lsi'].copy()

In [25]:
import pandas as pd

df_omics1 = pd.DataFrame(adata_omics1.X, index=adata_omics1.obs_names)
df_omics2 = pd.DataFrame(adata_omics2.X, index=adata_omics2.obs_names)
common_samples = df_omics1.index.intersection(df_omics2.index)
adata_omics2 = adata_omics2[adata_omics2.obs.index.isin(common_samples)]
adata_omics1 = adata_omics1[adata_omics1.obs.index.isin(common_samples)]

In [27]:
from SpatialGlue.preprocess import construct_neighbor_graph
data = construct_neighbor_graph(adata_omics1, adata_omics2, datatype=data_type)

/opt/anaconda3/envs/SpatialGlue/lib/python3.8/site-packages/SpatialGlue/preprocess.py:41: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata_omics1.uns['adj_spatial'] = adj_omics1
/opt/anaconda3/envs/SpatialGlue/lib/python3.8/site-packages/SpatialGlue/preprocess.py:46: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata_omics2.uns['adj_spatial'] = adj_omics2


In [29]:
from SpatialGlue.SpatialGlue_pyG import Train_SpatialGlue

In [31]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [33]:
model = Train_SpatialGlue(data, datatype=data_type, device=device)

/opt/anaconda3/envs/SpatialGlue/lib/python3.8/site-packages/SpatialGlue/preprocess.py:132: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:643.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [34]:
# train model
output = model.train()

  0%|                                                   | 0/200 [00:00<?, ?it/s]/opt/anaconda3/envs/SpatialGlue/lib/python3.8/site-packages/SpatialGlue/model.py:212: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.alpha = F.softmax(torch.squeeze(self.vu) + 1e-6)
100%|█████████████████████████████████████████| 200/200 [00:16<00:00, 12.40it/s]

Model training finished!



In [69]:
adata = adata_omics1.copy()
adata.obsm['emb_latent_omics1'] = output['emb_latent_omics1']
adata.obsm['emb_latent_omics2'] = output['emb_latent_omics2']
adata.obsm['SpatialGlue'] = output['SpatialGlue']
adata.obsm['alpha'] = output['alpha']
adata.obsm['alpha_omics1'] = output['alpha_omics1']
adata.obsm['alpha_omics2'] = output['alpha_omics2']

In [43]:
# we set 'mclust' as clustering tool by default. Users can also select 'leiden' and 'louvain'
from SpatialGlue.utils import clustering
tool = 'mclust' # mclust, leiden, and louvain
clustering(adata, key='SpatialGlue', add_key='SpatialGlue', n_clusters=18, method=tool, use_pca=True, start=0.9, end=1.1, increment=0.02)

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


## GraphGBM

In [44]:
from sklearn.decomposition import PCA
# from GraphST.utils import refine_label
from sklearn.preprocessing import StandardScaler
# from GraphST.utils import mclust_R
import numpy as np
from numpy import dot, array
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.mixture import BayesianGaussianMixture
from mvlearn.datasets import load_UCImultifeature
from mvlearn.embed import GCCA

from mvlearn.plotting import crossviews_plot
from mvlearn.embed import DCCA
from mvlearn.embed import MCCA
from mvlearn.embed import KMCCA

In [47]:
embedding = adata_omics1.obsm['feat']
n_neighbors = 4
connectivity = kneighbors_graph(adata_omics1.obsm['spatial'], n_neighbors=n_neighbors, include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)
embedding_RNA = connectivity.dot(embedding)
adata_omics1.obsm['spatial_RNAfeat'] = embedding_RNA

embedding = adata_omics2.obsm['feat']
embedding_Pro = connectivity.dot(embedding)
adata_omics1.obsm['spatial_Profeat'] = embedding_Pro

# Standardize the data
scaler_a = StandardScaler()
scaler_b = StandardScaler()

data_a_train = scaler_a.fit_transform(embedding_RNA)
data_b_train = scaler_b.fit_transform(embedding_Pro)

# Define and train the CCA model
n_components = 5  # Number of canonical components
cca = CCA(n_components=n_components)
cca.fit(data_a_train, data_b_train)

CCA(n_components=5)

In [49]:
### SOTA
from sklearn.mixture import BayesianGaussianMixture
from mvlearn.datasets import load_UCImultifeature
from mvlearn.embed import GCCA

from mvlearn.plotting import crossviews_plot
from mvlearn.embed import DCCA
from mvlearn.embed import MCCA
from mvlearn.embed import KMCCA

Xs = [data_a_train-data_a_train.min(), data_b_train-data_b_train.min()] # multiview data
mcca = KMCCA(n_components = 20, kernel = 'poly', regs = 1)
mcca.fit(Xs)

Xs_latents = mcca.transform(Xs)
adata_omics1.obsm['emb_pca'] = np.concatenate((Xs_latents[0,:,:], Xs_latents[1,:,:]), axis=1)

In [50]:
# GraphBGM: use BayesianGaussianMixture
gmm = BayesianGaussianMixture(n_components=18, covariance_type='full', random_state=42, init_params = 'random_from_data', n_init = 5, max_iter = 1000)

# Step 4: Fit GMM
gmm.fit(adata_omics1.obsm['emb_pca'])
cluster_labels = gmm.predict(adata_omics1.obsm['emb_pca'])
refine_cluster_labels = cKD_refine_label(np.array(adata_omics1.obsm['spatial']), cluster_labels, k = 45)
adata_omics1.obs['GraphBGM'] = refine_cluster_labels

In [61]:
from esda.moran import Moran
from libpysal.weights import DistanceBand
import numpy as np
import pandas as pd

In [79]:
bgm_pred = adata_omics1.obs['GraphBGM']
spatialglue_pred = adata.obsm['SpatialGlue']

In [95]:
import numpy as np
import pandas as pd
from libpysal.weights import DistanceBand
from esda.moran import Moran

results = []

# Lấy spatialglue_pred_dim0
spatialglue_pred_dim0 = spatialglue_pred[:, 0]  # shape (9196,)

# Khai báo phương pháp và tên tương ứng
methods = [bgm_pred, spatialglue_pred_dim0]
names = ['GraphBGM', 'SpatialGlue']

coords = np.array(adata.obsm['spatial'])[:, :2]

# Lặp qua hai phương pháp
for name, method in zip(names, methods):
    labels = np.array(method).astype(float)

    w = DistanceBand(coords, threshold=10, binary=True)

    moran = Moran(labels, w)

    results.append({
        'Method': name,    # Ghi tên (GraphBGM hoặc SpatialGlue)
        'Moran_I': moran.I,
    })

# Chuyển thành DataFrame
df_moran = pd.DataFrame(results)
print(df_moran)
df_moran.to_csv('brain_moran_results_ATAC.csv', index=False)

        Method   Moran_I
0     GraphBGM  0.698833
1  SpatialGlue  0.232364
